In [210]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_glpk ()
    helper.install_idaes()
    helper.install_ipopt()
!pip install requests folium
!pip install openrouteservice

In [211]:
import pandas as pd

df_clients = pd.read_csv('./content/caso3/clients.csv')
df_depots = pd.read_csv('./content/caso3/depots.csv')
df_vehicles = pd.read_csv('./content/caso3/vehicles.csv')

In [212]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [213]:
precio_km = 3032.1

# Leer el CSV
df = pd.read_csv('distances_all.csv')

# Inicializar ambos diccionarios
distance = {}
cost     = {}

# Recorrer una única vez (usando zip para eficiencia)
for ori, dst, d in zip(df['origen'], df['destino'], df['distancia_km']):
    key = (ori, dst)
    distance[key] = d
    cost[key]     = d * precio_km

# Verificar
print(distance[('CD1','CD2')], cost[('CD1','CD2')])

32.99 100028.979


In [214]:
depot_capacity = {}
for depot, d in zip(df_depots['DepotID'], df_depots['Capacity']):
    depot_capacity[f'CD{depot}'] = d

print(depot_capacity)

demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c

print(demand)

vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c

print(vehicle_capacity)

vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

print(vehicle_range)


{'CD1': 11, 'CD2': 90, 'CD3': 130, 'CD4': 145, 'CD5': 260, 'CD6': 180, 'CD7': 720, 'CD8': 55, 'CD9': 70, 'CD10': 75, 'CD11': 90, 'CD12': 270}
{'C1': 12, 'C2': 12, 'C3': 12, 'C4': 12, 'C5': 12, 'C6': 12, 'C7': 12, 'C8': 12, 'C9': 12, 'C10': 12, 'C11': 12, 'C12': 12, 'C13': 8, 'C14': 12, 'C15': 12, 'C16': 12, 'C17': 12, 'C18': 12, 'C19': 12, 'C20': 12, 'C21': 12, 'C22': 12, 'C23': 12, 'C24': 12, 'C25': 12, 'C26': 12, 'C27': 12, 'C28': 12, 'C29': 12, 'C30': 12, 'C31': 11, 'C32': 12, 'C33': 12, 'C34': 12, 'C35': 12, 'C36': 12, 'C37': 12, 'C38': 12, 'C39': 12, 'C40': 12, 'C41': 12, 'C42': 12, 'C43': 12, 'C44': 12, 'C45': 12, 'C46': 12, 'C47': 12, 'C48': 12, 'C49': 12, 'C50': 12, 'C51': 12, 'C52': 12, 'C53': 12, 'C54': 12, 'C55': 12, 'C56': 12, 'C57': 9, 'C58': 12, 'C59': 12, 'C60': 12, 'C61': 12, 'C62': 12, 'C63': 12, 'C64': 12, 'C65': 12, 'C66': 12, 'C67': 12, 'C68': 12, 'C69': 12, 'C70': 12, 'C71': 12, 'C72': 12, 'C73': 12, 'C74': 12, 'C75': 12, 'C76': 12, 'C77': 12, 'C78': 12, 'C79': 12,

In [255]:
# Etiquetas
depot_ids    = [str(d) for d in df_depots['DepotID']]
depot_labels = [f"CD{d}" for d in depot_ids]
client_ids   = [str(c) for c in df_clients['ClientID']]
client_labels= [f"C{c}"  for c in client_ids]
vehicle_list = df_vehicles['VehicleName'].tolist()

In [256]:
unassigned = set(client_labels)
clients_per_depot = {d: [] for d in depot_labels}

# radios a probar:
radii = sorted({
    dist
    for (d, c), dist in distance.items()
    if d in depot_labels and c in client_labels
})

for R in radii:

    for d in depot_labels:
        total_demand = sum(demand[c] for c in clients_per_depot[d])
        available_capacity = depot_capacity[d] - total_demand  

        if available_capacity <= 0:
            continue 

        for c in list(unassigned):
            if distance[(d, c)] <= R and demand[c] <= available_capacity:
                clients_per_depot[d].append(c)
                unassigned.remove(c)
                available_capacity -= demand[c] 
        if not unassigned:
            break 

    if not unassigned:
        break  # si todos los clientes están asignados, salir del ciclo de los radios

# 4) Resultado
for d, clist in clients_per_depot.items():
    print(f"{d} atiende a {len(clist)} clientes: {clist}")


CD1 atiende a 0 clientes: []
CD2 atiende a 0 clientes: []
CD3 atiende a 5 clientes: ['C31', 'C77', 'C5', 'C6', 'C2']
CD4 atiende a 12 clientes: ['C7', 'C67', 'C74', 'C58', 'C66', 'C69', 'C4', 'C47', 'C20', 'C63', 'C41', 'C82']
CD5 atiende a 14 clientes: ['C48', 'C16', 'C45', 'C18', 'C3', 'C51', 'C28', 'C17', 'C60', 'C46', 'C76', 'C65', 'C19', 'C42']
CD6 atiende a 13 clientes: ['C55', 'C30', 'C85', 'C26', 'C32', 'C57', 'C11', 'C1', 'C13', 'C75', 'C64', 'C87', 'C33']
CD7 atiende a 17 clientes: ['C34', 'C39', 'C15', 'C59', 'C56', 'C23', 'C24', 'C84', 'C35', 'C43', 'C73', 'C89', 'C10', 'C90', 'C14', 'C12', 'C72']
CD8 atiende a 3 clientes: ['C44', 'C62', 'C78']
CD9 atiende a 5 clientes: ['C9', 'C49', 'C61', 'C29', 'C21']
CD10 atiende a 6 clientes: ['C53', 'C25', 'C83', 'C52', 'C80', 'C86']
CD11 atiende a 5 clientes: ['C38', 'C50', 'C40', 'C71', 'C81']
CD12 atiende a 10 clientes: ['C79', 'C22', 'C68', 'C70', 'C37', 'C54', 'C27', 'C88', 'C36', 'C8']


In [257]:
from statistics import mean
import folium
# ————— 1. Mapear etiquetas a coordenadas —————
depot_coords = { f"CD{i+1}": coord_depots[i] 
                 for i in range(len(coord_depots)) }
client_coords = { f"C{i+1}": coord_clients[i] 
                  for i in range(len(coord_clients)) }

# ————— 2. Centro del mapa —————
all_lons = [lon for lon, lat in coord_depots] + [lon for lon, lat in coord_clients]
all_lats = [lat for lon, lat in coord_depots] + [lat for lon, lat in coord_clients]
map_center = ( mean(all_lats), mean(all_lons) )

# ————— 3. Colores para cada depósito —————
palette = [
    'blue','green','purple','orange','cadetblue','darkred',
    'darkgreen','darkpurple','gray','lightblue','pink','black'
]
colors = {
    depot: palette[i % len(palette)]
    for i, depot in enumerate(depot_coords)
}

# ————— 4. Crear mapa Folium —————
m = folium.Map(location=map_center, zoom_start=12)

# 4.2. Añadir clientes coloreados según su depósito asignado
for depot, client_list in clients_per_depot.items():
    col = colors[depot]
    for client in client_list:
        lon, lat = client_coords[client]
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            color=col,
            fill=True,
            fill_color=col,
            fill_opacity=0.7,
            popup=f"{client}, dem={demand[client]}"
        ).add_to(m)
        
# 4.1. Añadir depósitos con estrella de color según su asignación
for depot, (lon, lat) in depot_coords.items():
    col = colors[depot]   # color idéntico al de sus clientes
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(icon='star', prefix='fa', color=col),
        popup=f"{depot}, cap={depot_capacity[depot]}"
    ).add_to(m)

m

In [258]:
assign_client = {
    c: d
    for d, clist in clients_per_depot.items()
    for c in clist
}

In [259]:
assign_client

{'C31': 'CD3',
 'C77': 'CD3',
 'C5': 'CD3',
 'C6': 'CD3',
 'C2': 'CD3',
 'C7': 'CD4',
 'C67': 'CD4',
 'C74': 'CD4',
 'C58': 'CD4',
 'C66': 'CD4',
 'C69': 'CD4',
 'C4': 'CD4',
 'C47': 'CD4',
 'C20': 'CD4',
 'C63': 'CD4',
 'C41': 'CD4',
 'C82': 'CD4',
 'C48': 'CD5',
 'C16': 'CD5',
 'C45': 'CD5',
 'C18': 'CD5',
 'C3': 'CD5',
 'C51': 'CD5',
 'C28': 'CD5',
 'C17': 'CD5',
 'C60': 'CD5',
 'C46': 'CD5',
 'C76': 'CD5',
 'C65': 'CD5',
 'C19': 'CD5',
 'C42': 'CD5',
 'C55': 'CD6',
 'C30': 'CD6',
 'C85': 'CD6',
 'C26': 'CD6',
 'C32': 'CD6',
 'C57': 'CD6',
 'C11': 'CD6',
 'C1': 'CD6',
 'C13': 'CD6',
 'C75': 'CD6',
 'C64': 'CD6',
 'C87': 'CD6',
 'C33': 'CD6',
 'C34': 'CD7',
 'C39': 'CD7',
 'C15': 'CD7',
 'C59': 'CD7',
 'C56': 'CD7',
 'C23': 'CD7',
 'C24': 'CD7',
 'C84': 'CD7',
 'C35': 'CD7',
 'C43': 'CD7',
 'C73': 'CD7',
 'C89': 'CD7',
 'C10': 'CD7',
 'C90': 'CD7',
 'C14': 'CD7',
 'C12': 'CD7',
 'C72': 'CD7',
 'C44': 'CD8',
 'C62': 'CD8',
 'C78': 'CD8',
 'C9': 'CD9',
 'C49': 'CD9',
 'C61': 'CD9',
 'C

In [264]:
vehicles_per_depot  = {d: [] for d in depot_labels}
assigned_vehicles   = set()
unassigned_vehicles = set(vehicle_list)

for d in depot_labels:
    clientes = clients_per_depot.get(d, [])
    if not clientes:
        continue

    # 1) Demandas y capacidades
    total_demand = sum(demand[c] for c in clientes)
    if total_demand > depot_capacity.get(d, 0):
        print(f"Depósito {d}: capacidad depósito {depot_capacity[d]} < demanda {total_demand}")

    # 2) Estimación de distancia total (ida y vuelta a cada cliente)
    total_dist_req = 2 * sum(distance[(d, c)] for c in clientes)

    cap_acum   = 0.0
    range_acum = 0.0

    # 3) Tomamos vehículos aún libres, ordenados por capacidad descendente
    candidatos = sorted(unassigned_vehicles,
                        key=lambda v: vehicle_capacity[v],
                        reverse=True)

    for v in candidatos:
        if cap_acum >= total_demand and range_acum >= total_dist_req:
            break

        vehicles_per_depot[d].append(v)
        assigned_vehicles.add(v)
        unassigned_vehicles.remove(v)

        cap_acum   += vehicle_capacity[v]
        range_acum += vehicle_range[v]

# 5) Mostrar asignaciones
for d, vs in vehicles_per_depot.items():
    if vs:
        print(f"{d} tiene asignados {len(vs)} vehículos (cap sum={sum(vehicle_capacity[v] for v in vs)}, " +
              f"range sum={sum(vehicle_range[v] for v in vs):.1f} km): {vs}")



CD3 tiene asignados 1 vehículos (cap sum=158, range sum=174.0 km): ['V4']
CD4 tiene asignados 2 vehículos (cap sum=268, range sum=342.0 km): ['V2', 'V1']
CD5 tiene asignados 2 vehículos (cap sum=251, range sum=1048.0 km): ['V7', 'V11']
CD6 tiene asignados 2 vehículos (cap sum=224, range sum=310.0 km): ['V3', 'V5']
CD7 tiene asignados 2 vehículos (cap sum=208, range sum=245.0 km): ['V6', 'V10']
CD8 tiene asignados 1 vehículos (cap sum=98, range sum=716.0 km): ['V12']
CD9 tiene asignados 1 vehículos (cap sum=96, range sum=160.0 km): ['V9']
CD10 tiene asignados 1 vehículos (cap sum=86, range sum=1023.0 km): ['V13']
CD11 tiene asignados 1 vehículos (cap sum=85, range sum=942.0 km): ['V15']
CD12 tiene asignados 2 vehículos (cap sum=153, range sum=1210.0 km): ['V8', 'V14']


In [261]:
from pyomo.environ import (ConcreteModel, Set, Param, Var, Objective, Constraint,
                           Binary, NonNegativeReals, minimize, value)
from pyomo.opt import SolverFactory

model = ConcreteModel()

# 2.1) Conjuntos
model.D = Set(initialize=depot_labels)
model.C = Set(initialize=client_labels)
model.V = Set(initialize=vehicle_list)
model.N = model.D | model.C

# 2.2) Parámetros de costo y distancia (ya def. en tu script)
model.cost     = Param(model.N, model.N, initialize=lambda m,i,j: cost[(i,j)],     within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda m,i,j: distance[(i,j)], within=NonNegativeReals)

# 2.3) Capacidades y rangos
model.depot_capacity   = Param(model.D, initialize=lambda m,d: depot_capacity[d])
model.vehicle_capacity = Param(model.V, initialize=lambda m,v: vehicle_capacity[v])
model.vehicle_range    = Param(model.V, initialize=lambda m,v: vehicle_range[v])
model.demand           = Param(model.C, initialize=lambda m,c: demand[c])

# 2.4) Variables
# y[v,d]=1 si v parte del depósito d
model.y = Var(model.V, model.D, domain=Binary)
# x[v,i,j]=1 si v viaja de i a j
model.x = Var(model.V, model.N, model.N, domain=Binary)
# MTZ
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# 2.5) Objetivo: minimizar costo total
def obj_rule(m):
    return sum(m.cost[i,j] * m.x[v,i,j]
               for v in m.V for i in m.N for j in m.N if i!=j)
model.obj = Objective(rule=obj_rule, sense=minimize)

# ──────────────
# 2.6) Restricciones
# a) cada vehículo va a lo sumo a un depósito  
def one_depot_per_vehicle(m, v):
    return sum(m.y[v,d] for d in m.D) <= 1
model.one_depot = Constraint(model.V, rule=one_depot_per_vehicle)

# b) si v está asignado a d, sale exactamente una vez de d; si no, no sale  
def start_route(m, v, d):
    return sum(m.x[v,d,j] for j in m.N if j!=d) == m.y[v,d]
model.start_route = Constraint(model.V, model.D, rule=start_route)

# c) igual para el regreso  
def return_route(m, v, d):
    return sum(m.x[v,i,d] for i in m.N if i!=d) == m.y[v,d]
model.return_route = Constraint(model.V, model.D, rule=return_route)

# d) conservación de flujo en clientes  
def flow_cons(m, v, i):
    if i in m.C:
        return ( sum(m.x[v,j,i] for j in m.N if j!=i)
               == sum(m.x[v,i,j] for j in m.N if j!=i) )
    return Constraint.Skip
model.flow_cons = Constraint(model.V, model.N, rule=flow_cons)

def only_own_clients(m, v, i, j):
    # 1) Depósito → Depósito: no imponemos nada
    if i in m.D and j in m.D:
        return Constraint.Skip

    # 2) Arco que termina en cliente j: 
    #    solo si v partió de su depósito asignado
    if j in m.C:
        d_j = assign_client[j]
        return m.x[v, i, j] <= m.y[v, d_j]

    # 3) Arco que sale de cliente i → depósito j:
    if i in m.C and j in m.D:
        d_i = assign_client[i]
        return m.x[v, i, j] <= m.y[v, d_i]

    # 4) Cliente → Cliente: MTZ + flujo se encargan, aquí skip
    return Constraint.Skip

model.only_own = Constraint(model.V, model.N, model.N, rule=only_own_clients)


# f) cada cliente lo visita exactamente un vehículo
def each_client_once(m, c):
    return sum(m.x[v,j,c] for v in m.V for j in m.N if j!=c) == 1
model.visit_once = Constraint(model.C, rule=each_client_once)

# g) capacidad vehículo (si no está asignado, y[v,d]==0 → x=0 → carga=0)  
def vehicle_cap(m, v):
    return sum( m.demand[i] * sum(m.x[v,j,i] for j in m.N if j!=i)
               for i in m.C ) <= sum(m.y[v,d]*m.vehicle_capacity[v] for d in m.D)
model.veh_cap = Constraint(model.V, rule=vehicle_cap)

# h) capacidad depósito: suma de demandas servidas desde d ≤ cap[d]  
# (opcional) pre-extrae las listas para que queden como depot_clients

depot_clients = clients_per_depot

def depot_cap(m, d):
    if not depot_clients[d]:  # si no hay clientes asignados al depósito
        return Constraint.Skip

    return sum(
        m.demand[i] * sum(m.x[v, j, i] for j in m.N if j != i)
        for v in m.V
        for i in depot_clients[d]
    ) <= m.depot_capacity[d]

model.dep_cap = Constraint(model.D, rule=depot_cap)

# i) rango vehículo  
def veh_range(m, v):
    return sum(m.distance[i,j]*m.x[v,i,j] for i in m.N for j in m.N if i!=j) \
           <= sum(m.y[v,d]*m.vehicle_range[v] for d in m.D)
model.veh_range = Constraint(model.V, rule=veh_range)

# j) MTZ subtours (igual que antes)
def mtz_rule(m, v, i, j):
    if i!=j:
        return (m.u[v,i] - m.u[v,j]
               + m.vehicle_capacity[v]*m.x[v,i,j]
               <= m.vehicle_capacity[v] - m.demand[j])
    return Constraint.Skip
model.mtz     = Constraint(model.V, model.C, model.C, rule=mtz_rule)
model.mtz_bnd = Constraint(model.V, model.C,
                           rule=lambda m,v,i: (m.demand[i], m.u[v,i], m.vehicle_capacity[v]))

# k) satisfacción de demanda parcial (MTZ auxiliar)
def demand_sat(m, v, i):
    return m.u[v,i] >= m.demand[i] * sum(m.x[v,j,i] for j in m.N if j!=i)
model.demand_sat = Constraint(model.V, model.C, rule=demand_sat)

assign_vehicle= vehicles_per_depot
# Restricción: solo permitir y[v,d] si está en assign_vehicle[d]
def respect_vehicle_assignment(m, v, d):
    if v not in assign_vehicle.get(d, []):
        return m.y[v, d] == 0  # No permitido
    return Constraint.Skip
model.vehicle_assign_limit = Constraint(model.V, model.D, rule=respect_vehicle_assignment)




In [262]:
def print_routes(model):
    for v in model.V:
        print(f"\n>>> Ruta del vehículo {v}:")
        
        # 1. Encontrar el depósito asignado al vehículo y su capacidad
        depot = None
        depot_cap = 0
        for d in model.D:
            if value(model.y[v, d]) > 0.5:
                depot = d
                depot_cap = depot_capacity[d]
                break
        
        if depot is None:
            print("Vehículo no asignado a ningún depósito.")
            continue
        
        print(f" - Depósito de salida: {depot} (Capacidad: {depot_cap})")
        print(f" - Capacidad del vehículo: {vehicle_capacity[v]}")
        
        # 2. Reconstruir la ruta y calcular carga
        current_node = depot
        route = [current_node]
        visited = set()
        total_load = 0
        deliveries = {}  # Diccionario para guardar entregas entre nodos
        
        while True:
            next_node = None
            for j in model.N:
                if j != current_node and value(model.x[v, current_node, j]) > 0.5:
                    next_node = j
                    break
            
            if next_node is None or next_node in visited:
                break
            
            # Registrar entrega si es a un cliente
            if next_node in model.C:
                delivery = demand[next_node]
                total_load += delivery
                deliveries[(current_node, next_node)] = delivery
            
            route.append(next_node)
            visited.add(next_node)
            current_node = next_node
        
        # 3. Imprimir resultados
        if len(route) > 1:
            print(f" - Ruta: {' → '.join(str(node) for node in route)}")
            print(f" - Carga total transportada: {total_load}/{vehicle_capacity[v]}")
            
            # Imprimir entregas entre pares de nodos
            print(" - Entregas entre nodos:")
            for (i, j), qty in deliveries.items():
                print(f"    De {i} → {j}: Entrega {qty} unidades")
        else:
            print("Vehículo no utilizado.")


In [ ]:
from amplpy import modules

solver_name = "gurobi" 
solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
solver.options['TimeLimit'] = 500
solver.options['MIPGap']   = 0.01

res = solver.solve(model, load_solutions=True, tee=True)

from pyomo.opt import TerminationCondition
if res.solver.termination_condition not in (
    TerminationCondition.optimal,
    TerminationCondition.feasible):
    raise RuntimeError(f"No factible/óptimo: {res.solver.termination_condition}")

print_routes(model)


Gurobi 12.0.1:   lim:time = 500
  mip:gap = 0.01

>>> Ruta del vehículo V1:
 - Depósito de salida: CD4 (Capacidad: 145)
 - Capacidad del vehículo: 132
 - Ruta: CD4 → C7 → CD4
 - Carga total transportada: 12/132
 - Entregas entre nodos:
    De CD4 → C7: Entrega 12 unidades

>>> Ruta del vehículo V2:
 - Depósito de salida: CD4 (Capacidad: 145)
 - Capacidad del vehículo: 136
 - Ruta: CD4 → C67 → C74 → C69 → C20 → C41 → C82 → C63 → C4 → C47 → C66 → C58 → CD4
 - Carga total transportada: 132/136
 - Entregas entre nodos:
    De CD4 → C67: Entrega 12 unidades
    De C67 → C74: Entrega 12 unidades
    De C74 → C69: Entrega 12 unidades
    De C69 → C20: Entrega 12 unidades
    De C20 → C41: Entrega 12 unidades
    De C41 → C82: Entrega 12 unidades
    De C82 → C63: Entrega 12 unidades
    De C63 → C4: Entrega 12 unidades
    De C4 → C47: Entrega 12 unidades
    De C47 → C66: Entrega 12 unidades
    De C66 → C58: Entrega 12 unidades

>>> Ruta del vehículo V3:
 - Depósito de salida: CD6 (Capacida